# 路由链[Dynamically route logic based on input](https://python.langchain.com/docs/expression_language/how_to/routing)

 本节介绍LCEL中如何进行路由(routing)来创建非确定性链（链中下一步要执行的操作或流程依赖于前一步的输出，并不完全确定）。

  举个简单的例子，假设我们要构建一个问答系统。在用户输入一个问题后，我们首先需要判断这个问题是关于自然语言还是计算机视觉，然后根据分类的结果，将问题转发给不同的子系统进行解答。

  在这个场景中，第二步要执行的操作(转发给语言子系统或视觉子系统)就不能提前确定，它依赖于第一步的分类输出，所以这个链就是一个非确定性链。引入非确定性，可以让我们的系统有更多的灵活性，同时保证整个交互过程的一致性和结构性。而路由机制就是用来在这个非确定性链中明确定义执行流程的。


综上所述，路由允许链中的执行流程动态确定，为非确定性链提供一致性和结构，比如保证用户输入始终被正确处理。LCEL中两种执行路由有两种方法:

- 使用`RunnableBranch`
- 使用 `custom factory function`。该函数获取上一步的输入并返回一个`runnable`（重要的是返回一个`runnable`而不是直接执行）。
  

下面我们将使用两步序列来说明这两种方法，其中第一步对输入问题进行分类（`LangChain, Anthropic, or Other`），第二步根据分类结果路由到不同的`prompt`链。


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain.globals import set_debug
set_debug(False) 

## Using a RunnableBranch
`RunnableBranch`是一个可以根据条件选择运行不同分支的`Runnable`，现做一个最简示例：

In [2]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: isinstance(x, int), lambda x: x + 1),
    (lambda x: isinstance(x, float), lambda x: x * 2),
    lambda x: "goodbye",
)

branch.invoke("hello") # "HELLO"
branch.invoke(None) # "goodbye"


'goodbye'

代码中，我们定义了一个判断输入类型的条件函数和对应的处理`runnable`，以及一个默认的`runnable（lambda x: goodbye）`。在运行时，可以根据条件动态路由到不同分支的`Runnable`。

下面给出复杂一点的示例。首先，创建一个链，将传入的问题标识为 `LangChain` 、`Anthropic` 或 `Other` ：

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Anthropic`, or `Other`.
                                     
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatAnthropic()
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})    # 输出：' Anthropic'


d:\Users\DELL\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.anthropic.ChatAnthropic` was deprecated in langchain-community 0.0.28 and will be removed in 0.2. An updated version of the class exists in the langchain-anthropic package and should be used instead. To use it run `pip install -U langchain-anthropic` and import as `from langchain_anthropic import ChatAnthropic`.
  warn_deprecated(


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

现在，让我们创建三个子链：

- `langchain_chain`：让`ChatAnthropic()`用`LangChain`专家的身份回答问题，回答采用`As Harrison Chase told me`作为开头
- `anthropic_chain`：让`ChatAnthropic()`用`Anthropic`专家的身份回答问题，回答采用`As Dario Amodei told me`作为开头
- `general_chain`：直接回答问题


In [4]:
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic()
)

anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic()
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatAnthropic()
)


定义完整的路由链：

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)

full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch


- `branch`：使用`chain`的输出作为`topic`，根据其不同结果调用不同的分支
- `full_chain`：将`chain`和`branch`组合起来，完成问题分类和专家回答两个步骤。
  

在`full_chain`中，`{"topic": chain, "question": lambda x: x["question"]}`是一个字典形式的`Runnable`，这种写法本质上是使用了`RunnableParallel`，它将一个字典转换成并行执行的多个`Runnable`子链。当这个字典`Runnable`被执行时，它会：

- 并行执行`topic`和`question`两个支链
- 两个支链的结果会被合并为一个字典，然后做为一个整体输出，提供给后续的`branch Runnable`使用。


In [ ]:
# Anthropi分支
full_chain.invoke({"question": "how do I use Anthropic?"})

# LangChain分支
full_chain.invoke({"question": "how do I use LangChain?"})

# 其它分支
full_chain.invoke({"question": "whats 2 + 2"})


## 使用自定义函数
您还可以使用自定义函数在不同输出之间路由。下面是一个示例：

In [ ]:
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain


In [ ]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)


In [ ]:
full_chain.invoke({"question": "how do I use Anthroipc?"})
full_chain.invoke({"question": "how do I use LangChain?"})
full_chain.invoke({"question": "whats 2 + 2"})
